# SIMULATOR

__Notes__ code scripts at the bottom. Up the set to 2M with all assets. Set test set as 2017. 

In [1]:
import sys
paths = ['../scripts',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/quickpipeline',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/pd_feature_union']

for path in paths:
    sys.path.append(path)

from market_imports import *

from market_code import *
from market_trans import *
from quickpipe_mod import * 
from pandas_feature_union import *

data_path = '/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/data'

import warnings
warnings.filterwarnings(action='ignore')

# import datetime as dt




## Data Prep

In [2]:
csv = pd.read_csv(data_path + '/big.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

# COMPUTE RESIDUAL FOR RESPONSE
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose 

x = csv[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

csv['returnsOpenNextMktres10'] = result.resid
# Lose the first and last day
csv = csv[-csv['returnsOpenNextMktres10'].isnull()]
csv.drop(['returns_open_raw10_next'], axis=1, inplace=True)

check = csv.columns[:4].tolist()
csv.drop_duplicates(check, inplace=True)

# ELC is insanely high. The next highest price is around 300
csv[csv.Close ==csv.Close.max()]
csv[csv.Close > 1000]
csv = csv[csv.asset != 'ELC']

# Matching the kernel data some more
csv.index.name = 'time'
csv.reset_index(inplace=True)
csv.rename(columns={'asset': 'assetCode', 'Volume': 'volume',
                  'Close': 'close'}, inplace=True)

print(csv.shape)
csv.head()

(2121121, 10)


,time,assetCode,Open,close,volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returnsOpenNextMktres10
0,2013-01-16,EQC,16.00,16.049999,537700.0,0.004371,0.008158,-0.000623,-0.000625,-0.020582
1,2013-01-16,BDN,12.50,12.820000,2712500.0,0.004691,-0.003992,0.051211,0.020203,0.004975
2,2013-01-16,FBHS,31.00,30.870001,730700.0,-0.004202,-0.000967,0.004546,0.035794,-0.008944
3,2013-01-16,NMFC,14.65,14.740000,105100.0,0.003398,-0.009511,-0.016151,-0.023610,0.014740
4,2013-01-16,SMLP,19.67,19.660000,59000.0,0.001527,0.003565,-0.026106,-0.018137,-0.079554


In [3]:
test_csv = pd.read_csv(data_path + '/test.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

x = test_csv[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

test_csv['returnsOpenNextMktres10'] = result.resid
# Lose the first and last day
test_csv = test_csv[-test_csv['returnsOpenNextMktres10'].isnull()]
test_csv.drop(['returns_open_raw10_next'], axis=1, inplace=True)

check = test_csv.columns[:4].tolist()
test_csv.drop_duplicates(check, inplace=True)

# ELC is insanely high. The next highest price is around 300
test_csv[test_csv.Close ==test_csv.Close.max()]
test_csv[test_csv.Close > 1000]
test_csv = test_csv[test_csv.asset != 'ELC']

# Matching the kernel data some more
test_csv.index.name = 'time'
test_csv.reset_index(inplace=True)
test_csv.rename(columns={'asset': 'assetCode', 'Volume': 'volume',
                  'Close': 'close'}, inplace=True)

test_csv['date_'] = test_csv.time.dt.strftime('%Y%m%d').astype(int)
test_dates = test_csv['date_'].unique().tolist()
# the incoming test data will not have a response
y_test = test_csv['returnsOpenNextMktres10']
test_csv.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)

# SETTING UP THE TEST TRIAL
test_csv.set_index(['date_', 'assetCode'], inplace=True)
test_csv.sort_index(inplace=True)
print('\ntest data is set up as a sorted multi-index. This allows \nit \
to be used as an interator')

print(test_csv.shape)
test_csv.head()




test data is set up as a sorted multi-index. This allows 
it to be used as an interator
(527477, 8)


time       Open      close     volume  \
date_    assetCode                                               
20170118 A         2017-01-18  48.669998  48.250000  1523500.0   
         AA        2017-01-18  32.950001  34.810001  6420100.0   
         AAL       2017-01-18  47.400002  47.639999  5176500.0   
         AAN       2017-01-18  32.029999  32.310001   472200.0   
         AAON      2017-01-18  32.299999  32.200001   102700.0   

                    returns_close_raw  returns_open_raw  returns_close_raw10  \
date_    assetCode                                                             
20170118 A                  -0.001450          0.006183             0.037159   
         AA                  0.064366         -0.000607             0.188488   
         AAL                 0.018858         -0.003370             0.028531   
         AAN                 0.009016          0.000937             0.012771   
         AAON               -0.006192         -0.009245            -0.035091   

                    returns_open_raw10  
date_    assetCode                      
20170118 A                    0.057944  
         AA                   0.141585  
         AAL                  0.002535  
         AAN                 -0.011486  
         AAON                -0.034985

## Features 

In [4]:
from multiprocessing import Pool
from multiprocessing import cpu_count
cpu_count()

8

In [5]:
X = csv.copy()

In [6]:
def moving_average(x, n=20):
    # use cumsum()
    v = x.values
    v[n:] = v[n:] - v[:-n] 
    return v / n


def ewma_infinite_hist(arr_in, window):
    """Exponentialy weighted moving average specified by a decay ``window``
    assuming infinite history via the recursive form:

        (2) (i)  y[0] = x[0]; and
            (ii) y[t] = a*x[t] + (1-a)*y[t-1] for t>0.

    Parameters
    ----------
    arr_in : np.ndarray, float64
        A single dimenisional numpy array
    window : int64
        The decay window, or 'span'

    Returns
    -------
    np.ndarray
        The EWMA vector, same length / shape as ``arr_in``

    """
    n = arr_in.shape[0]
    ewma = np.empty(n)
    alpha = 2 / float(window + 1)
    ewma[0] = arr_in[0]
    for i in range(1, n):
        ewma[i] = arr_in[i] * alpha + ewma[i-1] * (1 - alpha)
    return ewma


def rsiVect(x, window=5): 
    ar = x.values
    ar[1:] = ar[1:] - ar[:-1]
    ar[0] = 0
    pos = ma.masked_where(ar < 0, ar).filled(0)
    neg = abs(ma.masked_where(ar > 0, ar).filled(0))

    x = ewma_infinite_hist(pos, window)
    y = ewma_infinite_hist(neg, window)

    return (x / (x+y))


def exTractTime(X, col, atts=list, mthds=None):

    mod = getattr(X, col)

    for i in range(len(atts)):
    
        X[atts[i]] = getattr(mod.dt, atts[i])
    
    while mthds:
        for i in range(len(mthds)):
    
            f = getattr(mod.dt, mthds[i])
            X[mthds[i]] = f()
        break
        
    return X


def toCatFeat(X, feats=list):
    for i in feats:
        colName = i + '_obj'
        X[colName] = X[i].astype(str) + '_'
    return X


def trainFeats(X, drops=list):
        
        feats = []
        for col in X.columns:
            if col not in drops:
                feats.append(col)
                
        return feats



In [7]:
def parallel(X, processes=cpu_count()):

    pool = Pool(processes)
    
    asset_close = X.groupby('assetCode')
    dframes = []
    for i in asset_close:
        df_ = i[1]    
        dframes.append(df_)
   
    df = pd.concat(pool.map(testFeatures, dframes))
    pool.close()
    pool.join()
    
    del dframes, asset_close
    
    df.sort_index(inplace=True)
    
    return df

In [9]:
def testFeatures(X):
    # should be cumsum
    v = X.close.cumsum().values
    n=5
#     v[:n] = np.nan
    v[n:] = (v[n:] - v[:-n]) / n
    X['sma'] = v
    X['rsi'] = rsiVect(X.close)
    
    del v
    
    exTractTime(X, 
                 col='time', 
                 atts=['weekday', 'month', 'year', 'day', 'quarter'],
                 mthds=['month_name'])
    
    toCatFeat(X, feats=['year', 'quarter'])
        
    return X

# def pipeWrap(X, feats=preproc_feats):

#     pipe = preprocPipe(X, binFeat='volume')
#     X_ = pipe.fit_transform(X[feats]).fillna(0)
#     return X_
    

In [ ]:
X = csv.copy()
X = parallel(X)

%time

In [ ]:
X.tail()

In [ ]:
X_ = X.copy()

In [ ]:
X_.head()

In [ ]:
preproc_feats = trainFeats(X_, drops=['time', 'assetCode', 'universe', 
                                    'returnsOpenNextMktres10', 
                                    'label'])

In [10]:
X_ = csv.copy()
X = parallel(X_)

X.dropna(inplace=True)
print('Dropped nulls from training set ONLY. \nFor testing, you must fill nulls')

# only use universe for real data simulation
X['universe'] = universe_feat(X) # N/A kernel
print('\nuniverse for simulation only \n')

# Create objects for ranges, etc
X['date_'] = X.time.dt.strftime("%Y%m%d").astype(int) 
dates = X['date_'].unique()
target = ['returnsOpenNextMktres10']
label = ['label']  
X[label[0]] = (X[target[0]] > 0).astype(int)
preproc_feats = trainFeats(X, drops=['date_', 'time', 'assetCode', 'universe', 
                                    'returnsOpenNextMktres10', 
                                    'label'])

test_pipe = preprocPipe(X, binFeat='volume')
X_p = test_pipe.fit_transform(X[preproc_feats])
X_p.head()

%time

Dropped nulls from training set ONLY. 
For testing, you must fill nulls

universe for simulation only 

pipeline created
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [11]:
X_p.head()

,Open,close,volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,sma,rsi,weekday,month,year,day,quarter,volume_bin_category_0.0,volume_bin_category_1.0,volume_bin_category_2.0,volume_bin_category_3.0,volume_bin_category_4.0,month_name_April,month_name_August,month_name_December,month_name_February,month_name_January,month_name_July,month_name_June,month_name_March,month_name_May,month_name_November,month_name_October,month_name_September,year_obj_2013_,year_obj_2014_,year_obj_2015_,year_obj_2016_,quarter_obj_1_,quarter_obj_2_,quarter_obj_3_,quarter_obj_4_
1960,-0.286948,0.000397,-0.264922,0.039750,0.076327,0.101209,0.082983,0.349045,1.866794,0.706062,-1.658757,-1.403675,0.148319,-1.387055,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1961,-1.321164,0.000150,-0.239368,0.058709,0.022123,0.093246,0.183386,-0.684093,1.866794,0.706062,-1.658757,-1.403675,0.148319,-1.387055,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1962,-1.155135,-0.000128,-0.318280,-0.020975,0.087219,0.699304,0.856020,-0.522091,-2.011027,0.706062,-1.658757,-1.403675,0.148319,-1.387055,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1963,0.894838,0.002771,3.584052,0.070042,-0.094893,-0.170835,-0.314676,1.558768,1.866794,0.706062,-1.658757,-1.403675,0.148319,-1.387055,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1964,-1.829349,0.001414,0.136833,0.725530,-0.084119,2.535926,2.353441,-1.153505,1.866794,0.706062,-1.658757,-1.403675,0.148319,-1.387055,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [12]:
# TRAIN THE MODEL
feats_train = X_p.columns.tolist()
print('List of training feats from the processed dataframe is created \n')

# Non training feats
feats = [col for col in X.columns if col not in preproc_feats]
print('List of non-training feats from the processed dataframe is created \n')

# Merge
X_ = pd.concat([X[feats] ,X_p],axis=1)
print('Non-training feats from unprocessed dframe and \ntraining feats from processed \
are merged. Shape: ', X_.shape)

List of training feats from the processed dataframe is created 

List of non-training feats from the processed dataframe is created 

Non-training feats from unprocessed dframe and 
training feats from processed are merged. Shape:  (2118754, 45)


In [14]:
# Kernel simulation
# Define the base ahead of time
base = csv.copy()
t = base.tail(1)['time'].tolist()[0]
print('Last date in the training data', t, '\n')
time = (t - dt.timedelta(days=10))
print('Past date needed for longest lag', time, '\n')

base_ = base[base.time > time]
# this could vary slightly in size

base_.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)
print('Base shape: ', base_.shape)

Last date in the training data 2016-12-15 00:00:00 

Past date needed for longest lag 2016-12-05 00:00:00 

Base shape:  (18166, 9)


In [16]:
# REFRESHER WALKTHROUGH
data_test = test_csv.copy()
# test = data_test.loc[test_dates[0]]
# test.reset_index(inplace=True)



In [17]:
env_sim = []
days = []
print_=True

for i in range(len(test_dates)):
    dframe = data_test.loc[test_dates[i]]
    days.append(dframe)
    
    

    # First day comes in...

    test = days[i]
    print('Day {} of {}'. format(i+1, len(test_dates)), '\n')
    if print_:
        print('Daily test shape: ', test.shape, '\n')
        print('Date: ', test.time[0], '\n')

    # Move assetCode from index to col
    test.reset_index(inplace=True)

    # Simulated predictions template with universe discrepencies
    vec = test.assetCode.tolist()
    template = pd.DataFrame({ 'assetCode' :random.sample(vec, k=int(len(vec)*.95)),
                            'confidenceValue': 0})   
    unv = template.assetCode.tolist()    
    df_unv = test[test.assetCode.isin(unv)]
    if print_:
        print('Template universe', len(unv), '\n')
        print('Simulated template unv reduction: ', len(vec) - int(len(vec)*.95), '\n')
        print('Daily test reduced for universe: ', df_unv.shape, '\n')

    # BASE
    # idenfify the oldest base date
    drop_time = base_.time.iloc[0]    
    # update the base with the test day
    base_ = pd.concat([base_, df_unv], axis=0)
    # drop the oldest date
    base_ = base_[base_.time != drop_time]
    # copy, just to be sure

    # ENGINEER FEATS
    X_test = parallel(base_)
    print('feats complete')
    

    if print_:
        print('X_test shape after processing: ',X_test.shape, '\n')
    print('drop off base')
    X_test = X_test.iloc[-df_unv.shape[0]:, :]

    if print_:
        print('Reduced: ',X_test.shape, '\n')

    # No longer able to drop nulls. Use fill instead

    preproc_feats = trainFeats(X_test, drops=['time', 'assetCode'])
    test_pipe = preprocPipe(X_test, binFeat='volume')

    X_p = test_pipe.fit_transform(X_test[preproc_feats]).fillna(0)


    if print_:
        print('Transformed and nulls filled \n')

    preds = model.predict(X_p, num_iteration=model.best_iteration) * 2 - 1
    if print_:
        print('pred shape: ', preds.shape, '\n')

    df_pred = pd.DataFrame({'assetCode':X_test['assetCode'],'conf':preds})
    pred_dict = df_pred.set_index('assetCode').to_dict()['conf']
    template['confidenceValue'] = template.assetCode.map(pred_dict)
    # template.head()

    env_sim.append(template)
    print('\nenv appended \n')
    del template

Day 1 of 231 

Daily test shape:  (2256, 8) 

Date:  2017-01-18 00:00:00 

Template universe 2143 

Simulated template unv reduction:  113 

Daily test reduced for universe:  (2143, 9) 

feats complete
X_test shape after processing:  (18038, 19) 

drop off base
Reduced:  (2143, 19) 

pipeline created
Transformed and nulls filled 

pred shape:  (2143,) 


env appended 

Day 2 of 231 

Daily test shape:  (2278, 8) 

Date:  2017-01-19 00:00:00 

Template universe 2164 

Simulated template unv reduction:  114 

Daily test reduced for universe:  (2164, 9) 

feats complete
X_test shape after processing:  (17924, 19) 

drop off base
Reduced:  (2164, 19) 

pipeline created
Transformed and nulls filled 

pred shape:  (2164,) 


env appended 

Day 3 of 231 

Daily test shape:  (2278, 8) 

Date:  2017-01-20 00:00:00 

Template universe 2164 

Simulated template unv reduction:  114 

Daily test reduced for universe:  (2164, 9) 

feats complete
X_test shape after processing:  (17811, 19) 

drop off

ValueError: shape mismatch: value array of shape (0,2166) could not be broadcast to indexing result of shape (1,2166)

In [1]:
env_sim[1]

NameError: name 'env_sim' is not defined

## All import functions

In [ ]:
####################################################################    
#                  TRANSFORMER                           #
####################################################################

from market_trans import *
from sklearn.pipeline import make_pipeline
import sys
paths = [
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/quickpipeline',
         '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/pd_feature_union' 
        ]

for path in paths:
    sys.path.append(path)
        
from quickpipe_mod import * 
from pandas_feature_union import *




def preprocPipe(X, binFeat):
    
    transformer_list = [ ('numeric', make_pipeline(TypeSelector(np.number),
                                QuickPipeline_mod()
                                )
                        ),('binned_features', make_pipeline(
                            TypeSelector(np.number),
                            SelectFeatures(feat_list=[binFeat]),
                            KBins(n_bins=5),
                            QuickPipeline_mod()
                             )
                        ), ('boolean_features', make_pipeline(
                                 TypeSelector(np.bool_),
                                QuickPipeline_mod(categorical_features=(TypeSelector(np.bool_).
                                                                        fit_transform(X).
                                                                        columns.
                                                                        tolist()
                                                                        ))
                                )
                       ),('categorical_features', make_pipeline(
                             TypeSelector(np.object),
                            QuickPipeline_mod() )
                            )                                           
                       ]
        
    pipe = make_pipeline(PandasFeatureUnion(transformer_list))
#     print('pipeline created')
    return pipe
    